In [43]:
import sync_wandb_run
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import utils as pu
from plot import create_plot_with_values
from plot_pretty import create_ultimate_plotly_plot, generate_scatter_plot, generate_scatter_plot_accuracy, generate_scatter_plot_gridscore
import plotly_utils

In [44]:
entity = "raphaelmaser"  # Your W&B username or team
project = "Improved-Interpretability-and-Concepts"  # Your W&B project name
df = sync_wandb_run.load_wandb_runs()
df.count()  # Display the number of runs loaded
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

val_accuracy = "Validation Accuracy"
grid_score = "PG Score"
frozen_layers = "config_define_frozen_layers"
trainable_params = "Trainable Parameters (M)"
time="Runtime (h)"
figsize=(10,4)


# Comparison between DoRA and LoRA

1. LoRA only applied to qkv projection without ReLU
2. DoRA only applied to qkv projection without ReLU

WD=0.1 for all

In [45]:

vitc = df[df['group'].isin(["vitc-lora-ranks", "vitc-dora-ranks"])] 
vitc["config_weight_decay"] = vitc["config_weight_decay"].astype(float)  # Ensure weight decay is float type
vitc = vitc[vitc["config_weight_decay"] == 0.1]  # Filter for weight decay 0.1
vit_base = df[df['id'] == "2fupnc3h"] # Base model
vit_conv_stem_only = df[df['id'] == "uaryeifu"]  # Conv stem only model
vitc_base = df[df['id'] == "n8dqbqan"]  # Conv stem only model
vitc = pd.concat([vit_base, vit_conv_stem_only, vitc, vitc_base], ignore_index=True)  # Combine base and conv stem only models

vitc.reset_index(drop=True, inplace=True)  # Reset index after filtering
print(f"Number of runs in layerwise_freezing group: {len(vitc)}")

#Renaming
vitc = pu.rename(vitc).sort_values(by=val_accuracy, ascending=False)
vitc = pu.rename_adapter_factor_experiments(vitc)  # Rename columns for adapter factor experiments
vitc.rename(columns={
    "config_lora_rank": "Lora Rank",
    "config_trainable_parameters": trainable_params,
    "summary__runtime": time,
}, inplace=True)

vitc.replace({
    "Model": {
        "vitc_b_16_B=2_pretrained_lora_kaiming": "ViTc LoRA (No ReLU, only qkv)",
        "vitc_b_16_B=2_pretrained_dora_kaiming": "ViTc DoRA (No ReLU, only qkv)",
        "vitc_b_16_B=2_pretrained_only-conv-stem": "ViTc - conv stem only",
        "vit_b_16_B=2_pretrained": "ViT",
        "vit_b_16_B=2_pretrained_conv-stem": "ViTc"
    }}, inplace=True)  # Rename specific model names for clarity

vitc[time] = vitc[time].apply(lambda x: x/(60*60))  # Format runtime to 2 decimal places
# Some stats
for name in vitc["Model"].unique(): print(name)  # Print each model name
display(vitc)


Number of runs in layerwise_freezing group: 15
ViTc
ViTc LoRA (No ReLU, only qkv)
ViTc - conv stem only
ViTc DoRA (No ReLU, only qkv)
ViT


/tmp/ipykernel_3328286/3854885760.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



id                          Model     state        entity  \
14  n8dqbqan                           ViTc  finished  raphaelmaser   
5   3m2yljdz  ViTc LoRA (No ReLU, only qkv)  finished  raphaelmaser   
11  fphpl0nv  ViTc LoRA (No ReLU, only qkv)  finished  raphaelmaser   
12  4g0k0tjg  ViTc LoRA (No ReLU, only qkv)  finished  raphaelmaser   
8   szxir6fn  ViTc LoRA (No ReLU, only qkv)  finished  raphaelmaser   
6   x0rv38z9  ViTc LoRA (No ReLU, only qkv)  finished  raphaelmaser   
9   p0y0kg5c  ViTc LoRA (No ReLU, only qkv)  finished  raphaelmaser   
1   uaryeifu          ViTc - conv stem only  finished  raphaelmaser   
7   iklv8i7m  ViTc DoRA (No ReLU, only qkv)  finished  raphaelmaser   
10  3gwra9hk  ViTc DoRA (No ReLU, only qkv)  finished  raphaelmaser   
2   jjeogw2n  ViTc DoRA (No ReLU, only qkv)  finished  raphaelmaser   
4   l59n3gtw  ViTc DoRA (No ReLU, only qkv)  finished  raphaelmaser   
3   2lgd4qop  ViTc DoRA (No ReLU, only qkv)  finished  raphaelmaser   
13  k5iwv3v2  ViTc DoRA (No ReLU, only qkv)  finished  raphaelmaser   
0   2fupnc3h                            ViT  finished  raphaelmaser   

                                   project            created_at  updated_at  \
14  Improved-Interpretability-and-Concepts  2025-07-19T06:49:42Z         NaN   
5   Improved-Interpretability-and-Concepts  2025-07-29T22:46:34Z         NaN   
11  Improved-Interpretability-and-Concepts  2025-07-29T21:40:31Z         NaN   
12  Improved-Interpretability-and-Concepts  2025-07-29T21:01:04Z         NaN   
8   Improved-Interpretability-and-Concepts  2025-07-29T21:01:04Z         NaN   
6   Improved-Interpretability-and-Concepts  2025-07-29T20:59:28Z         NaN   
9   Improved-Interpretability-and-Concepts  2025-07-29T20:51:04Z         NaN   
1   Improved-Interpretability-and-Concepts  2025-07-21T10:28:29Z         NaN   
7   Improved-Interpretability-and-Concepts  2025-07-29T20:43:36Z         NaN   
10  Improved-Interpretability-and-Concepts  2025-07-29T20:04:13Z         NaN   
2   Improved-Interpretability-and-Concepts  2025-07-29T20:43:37Z         NaN   
4   Improved-Interpretability-and-Concepts  2025-07-29T19:55:02Z         NaN   
3   Improved-Interpretability-and-Concepts  2025-07-29T20:48:08Z         NaN   
13  Improved-Interpretability-and-Concepts  2025-07-29T20:43:28Z         NaN   
0   Improved-Interpretability-and-Concepts  2025-07-19T06:48:52Z         NaN   

   tags notes                                                url  \
14  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
5   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
11  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
12  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
8   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
6   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
9   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
1   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
7   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
10  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
2   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
4   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
3   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
13  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
0   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   

              group  job_type  sweep                   synced_at  config_b  \
14    vit-conv-stem       NaN    NaN  2025-07-27T10:49:36.411784       2.0   
5   vitc-lora-ranks       NaN    NaN  2025-07-30T13:42:45.370527       2.0   
11  vitc-lora-ranks       NaN    NaN  2025-07-30T13:42:44.474280       2.0   
12  vitc-lora-ranks       NaN    NaN  2025-07-30T13:42:42.646008       2.0   
8   vitc-lora-ranks       NaN    NaN  2025-07-30T13:42:43.395933       2.0   
6   vitc-lora-ranks       NaN    Na

In [46]:
# vitc = pd.concat([vit_c, vitc], ignore_index=True)  # Add the qkv LoRA factors to the DataFrame

## Weight decay = 0.1

In [47]:

generate_scatter_plot_accuracy(
    data=vitc,
    x=trainable_params,
    y=val_accuracy,
    color="Model",
    scale_factor=2,
    title="ViTc - With Weight Decay (0.1)",
    connect_dots=True,
    scale_dots_by_trainable_params=trainable_params
)

In [50]:
generate_scatter_plot_gridscore(
    data=vitc,
    x=trainable_params,
    y=grid_score,
    color="Model",
    scale_factor=2,
    title="ViTc - With Weight Decay (0.1)",
    connect_dots=True,
    scale_dots_by_trainable_params=trainable_params
)

In [51]:
vit_base_wd_01= vitc[vitc['config_weight_decay'] == 0.1]  # Filter for weight decay 0.0

generate_scatter_plot_accuracy(
    data=vitc,
    x=trainable_params,
    y=time,
    color="Model",
    scale_factor=2,
    title="ViTc - With Weight Decay (0.1)",
    connect_dots=True,
    scale_dots_by_trainable_params=trainable_params,
    range_y=(0, 7)
)